In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
import datetime
import sys
import traceback
pd.set_option('display.max_column', 100)
import time

In [4]:
#load data to dataframe
df_itenary = pd.read_csv('../data/raw/itineraries_csv/ATL_full.csv', low_memory=False)

In [5]:
df_itenary

,legId,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,9ca0e81111c683bec1012473feefd28f,2022-04-16,2022-04-17,ATL,BOS,PT2H29M,False,False,True,248.6,947.0,1650214620,2022-04-17T12:57:00.000-04:00,1650223560,2022-04-17T15:26:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,8940,947,coach
1,98685953630e772a098941b71906592b,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,False,True,248.6,947.0,1650191400,2022-04-17T06:30:00.000-04:00,1650200400,2022-04-17T09:00:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9000,947,coach
2,98d90cbc32bfbb05c2fc32897c7c1087,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,False,True,248.6,947.0,1650209700,2022-04-17T11:35:00.000-04:00,1650218700,2022-04-17T14:05:00.000-04:00,BOS,ATL,Delta,DL,Boeing 757-200,9000,947,coach
3,969a269d38eae583f455486fa90877b4,2022-04-16,2022-04-17,ATL,BOS,PT2H32M,False,False,True,248.6,947.0,1650218340,2022-04-17T13:59:00.000-04:00,1650227460,2022-04-17T16:31:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9120,947,coach
4,980370cf27c89b40d2833a1d5afc9751,2022-04-16,2022-04-17,ATL,BOS,PT2H34M,False,False,True,248.6,947.0,1650203940,2022-04-17T09:59:00.000-04:00,1650213180,2022-04-17T12:33:00.000-04:00,BOS,ATL,Delta,DL,Airbus A321,9240,947,coach
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874739,dafe7cde105105c58119efad61a79151,2022-05-19,2022-06-04,ATL,SFO,PT8H40M,False,False,False,766.61,2838.0,1654373700||1654383900,2022-06-04T16:15:00.000-04:00||2022-06-04T19:0...,1654379100||1654404900,2022-06-04T17:45:00.000-04:00||2022-06-04T21:5...,MCO||SFO,ATL||MCO,Delta||Alaska Airlines,DL||AS,Boeing 757||Boeing 737-900,5400||21000,399||2439,coach||coach
874740,57a130a0b9af2de4383e3ce14888fed1,2022-05-19,2022-06-04,ATL,SFO,PT16H1M,False,False,False,812.21,3328.0,1654396500||1654431600,2022-06-04T22:35:00.000-04:00||2022-06-05T08:2...,1654404480||1654454160,2022-06-05T00:48:00.000-04:00||2022-06-05T11:3...,EWR||SFO,ATL||EWR,Delta||Alaska Airlines,DL||AS,Boeing 737-800||Boeing 737-900,7980||22560,762||2566,coach||coach
874741,021df79efd5a9adbc865403cba3bcaf1,2022-05-19,2022-06-04,ATL,SFO,PT8H50M,False,False,False,886.6,2857.0,1654344000||1654367700,2022-06-04T08:00:00.000-04:00||2022-06-04T11:3...,1654363500||1654375800,2022-06-04T10:25:00.000-07:00||2022-06-04T13:5...,SEA||SFO,ATL||SEA,Alaska Airlines||Alaska Airlines,AS||AS,Boeing 737 MAX 9||Airbus A320,19500||8100,2178||679,coach||coach
874742,88a4642c7f3edaf1a5a0be0c570ebdd8,2022-05-19,2022-06-04,ATL,SFO,PT9H55M,False,False,False,886.6,2857.0,1654344000||1654371900,2022-06-04T08:00:00.000-04:00||2022-06-04T12:4...,1654363500||1654379700,2022-06-04T10:25:00.000-07:00||2022-06-04T14:5...,SEA||SFO,ATL||SEA,Alaska Airlines||Alaska Airlines,AS||AS,Boeing 737 MAX 9||Boeing 737-800,19500||7800,2178||679,coach||coach


In [127]:
df_itenary_cp=df_itenary[['searchDate', 'flightDate', 'startingAirport',
       'destinationAirport', 'travelDuration', 'isBasicEconomy',
       'isRefundable', 'isNonStop', 'totalFare', 'totalTravelDistance',
        'segmentsDepartureTimeRaw','segmentsAirlineCode','segmentsCabinCode']].copy()

In [129]:
df_itenary_cp['totalTravelDistance'] = df_itenary_cp['totalTravelDistance'].fillna('PT0H0M')

In [134]:
df_itenary_cp['searchDate']=pd.to_datetime((df_itenary_cp['searchDate'].str.replace('-', '')), \
                                           format='%Y%m%d', errors='coerce')
df_itenary_cp['flightDate']=pd.to_datetime((df_itenary_cp['flightDate'].str.replace('-', '')), \
                                           format='%Y%m%d', errors='coerce')
df_itenary_cp['segmentsCabinCode']=df_itenary_cp['segmentsCabinCode'].str.replace('|',',').str.split(',').str[0]
df_itenary_cp['segmentsDepartureTimeRaw']=pd.to_datetime((df_itenary_cp['segmentsDepartureTimeRaw'] \
                                                          .str.replace('|',',').str.split(',').str[0]), \
                                                         format='%Y-%m-%dT%H:%M:%S.%f%z', errors='coerce')

df_itenary_cp['segmentsAirlineCode']=df_itenary_cp['segmentsAirlineCode'].str.replace('|',',').str.split(',').str[0]

In [138]:
df_itenary_cp=df_itenary_cp.dropna()

In [139]:
def convert_to_epoch():
    depart_time = []  # Initialize the list to store departure times
    depttime=df_itenary_cp['segmentsDepartureTimeRaw']
    for depttime in depttime :
        ts = pd.Timestamp(depttime)
        Hour = int(ts.hour)
        Minute = int(ts.minute)
        depart_time.append(f"{Hour:02d}:{Minute:02d}")  # Format the time as HH:MM
    df_itenary_cp['depart_time'] = depart_time # Create a DataFrame from the list
        
    return df_itenary_cp

In [140]:
df_itenary_cp=convert_to_epoch()

<ipython-input-139-84d1229e3494>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_itenary_cp['depart_time'] = depart_time # Create a DataFrame from the list


In [146]:
import re

def convert_travel_time():
    # Define the duration string
    duration_str = df_itenary_cp['travelDuration']
    total_minutes=[]
    for duration in duration_str:
        # Use regular expressions to extract hours and minutes
        hours_match = re.search(r'(\d+)H', duration)
        minutes_match = re.search(r'(\d+)M', duration)
        # Initialize variables for hours and minutes
        hours = 0
        minutes = 0
        # If the matches are found, convert them to integers
        if hours_match:
            hours = int(hours_match.group(1))
            if minutes_match:
                minutes = int(minutes_match.group(1))
        # Calculate the total duration in minutes
        total_minutes.append(hours * 60 + minutes)
    
    df_itenary_cp['travelDuration_minutes']=total_minutes
    return df_itenary_cp

In [147]:
df_itenary_cp.columns

Index(['searchDate', 'flightDate', 'startingAirport', 'destinationAirport',
       'travelDuration', 'isBasicEconomy', 'isRefundable', 'isNonStop',
       'totalFare', 'totalTravelDistance', 'segmentsDepartureTimeRaw',
       'segmentsAirlineCode', 'segmentsCabinCode', 'depart_time',
       'travelDuration_minutes'],
      dtype='object')

In [148]:
df_itenary_cp=df_itenary_cp[['searchDate', 'flightDate', 'startingAirport', 'destinationAirport',
       'travelDuration_minutes', 'isBasicEconomy', 'isRefundable', 'isNonStop',
       'totalFare', 'totalTravelDistance', 'depart_time',
       'segmentsAirlineCode', 'segmentsCabinCode',
       ]]

In [149]:
df_itenary_cp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 874069 entries, 0 to 874743
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   searchDate              874069 non-null  datetime64[ns]
 1   flightDate              874069 non-null  datetime64[ns]
 2   startingAirport         874069 non-null  object        
 3   destinationAirport      874069 non-null  object        
 4   travelDuration_minutes  874069 non-null  int64         
 5   isBasicEconomy          874069 non-null  object        
 6   isRefundable            874069 non-null  object        
 7   isNonStop               874069 non-null  object        
 8   totalFare               874069 non-null  object        
 9   totalTravelDistance     874069 non-null  object        
 10  depart_time             874069 non-null  object        
 11  segmentsAirlineCode     874069 non-null  object        
 12  segmentsCabinCode       874069 non-

In [150]:
df_itenary_cp.to_parquet('../data/processed/df_itenary_ATL.parquet', index=False)

,searchDate,flightDate,startingAirport,destinationAirport,travelDuration_minutes,isBasicEconomy,isRefundable,isNonStop,totalFare,totalTravelDistance,depart_time,segmentsAirlineCode,segmentsCabinCode
0,2022-04-16,2022-04-17,ATL,BOS,149,False,False,True,248.6,947.0,12:57,DL,coach
1,2022-04-16,2022-04-17,ATL,BOS,150,False,False,True,248.6,947.0,06:30,DL,coach
2,2022-04-16,2022-04-17,ATL,BOS,150,False,False,True,248.6,947.0,11:35,DL,coach
3,2022-04-16,2022-04-17,ATL,BOS,152,False,False,True,248.6,947.0,13:59,DL,coach
4,2022-04-16,2022-04-17,ATL,BOS,154,False,False,True,248.6,947.0,09:59,DL,coach
...,...,...,...,...,...,...,...,...,...,...,...,...,...
874739,2022-05-19,2022-06-04,ATL,SFO,520,False,False,False,766.61,2838.0,16:15,DL,coach
874740,2022-05-19,2022-06-04,ATL,SFO,961,False,False,False,812.21,3328.0,22:35,DL,coach
874741,2022-05-19,2022-06-04,ATL,SFO,530,False,False,False,886.6,2857.0,08:00,AS,coach
874742,2022-05-19,2022-06-04,ATL,SFO,595,False,False,False,886.6,2857.0,08:00,AS,coach


In [ ]:
destinationAirport